In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st

from dict_utils import *
from constants import *
from utils_api import *
from utils import *

pd.set_option('display.max_columns', None)

2024-05-23 20:41:28.001 
  command:

    streamlit run c:\Users\mikej\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


AttributeError: st.secrets has no attribute "dg_key". Did you forget to add it to secrets.toml or the app settings on Streamlit Cloud? More info: https://docs.streamlit.io/streamlit-cloud/get-started/deploy-an-app/connect-to-data-sources/secrets-management

In [ ]:
#### FILES

dg = "https://feeds.datagolf.com/preds/fantasy-projection-defaults?tour=pga&site=draftkings&slate=main&file_format=csv&key=e297e933c3ad47d71ec1626c299e"
fantrax = r"C:\Users\mikej\Desktop\fantrax\fantrax.csv"

In [ ]:
team_color={
     "Philly919": 'rgb(14,195,210)',
     "unit_circle": 'rgb(194,139,221)',
     "AlphaWired": 'rgb(247,160,93)',
     "Sneads Foot": 'rgb(70,214,113)',
     "New Team 4": 'rgb(247,94,56)',
     "Team Gamble": 'rgb(38,147,190)',
     "txmoonshine": 'rgb(219,197,48)',
     "Putt Pirates": 'rgb(115,112,106)'}

active_color={
    "Active":'rgb(146,146,143)',
    "Reserve":'rgb(220,222,202)'
    }

teams_dict = {'919':'Philly919','u_c':'unit_circle',
              'NT 4':'New Team 4','NT 8':'Sneads Foot',
              'txms':'txmoonshine','MG':'Team Gamble',
              'grrr':'Putt Pirates','[AW]':'AlphaWired'}

In [ ]:
teams = pd.read_csv(fantrax ,usecols=['Player','Status','Roster Status'])
teams[51:100]

In [ ]:
dg_proj = pd.read_csv(dg)
dg_proj[:50]

In [ ]:
#fx rosters

teams = pd.read_csv(fantrax ,usecols=['Player','Status','Roster Status'])
teams.columns = ['player','team','active_reserve']
teams_dict = {'919':'Philly919','u_c':'unit_circle','NT 8':'Sneads Foot','NT 4':'New Team 4','txms':'txmoonshine','MG':'Team Gamble','grrr':'Putt Pirates','[AW]':'AlphaWired'}
teams['team'] = teams.team.map(teams_dict)
teams.set_index('player',inplace=True)

# datagolf dk projections
dg_proj = pd.read_csv(dg)#,usecols=['player_name','proj_points_total'])
dg_proj = dg_proj[['player_name','proj_points_total']]
dg_proj.columns = ['player','proj_pts']

def fix_long_names(dg_proj):
        #flip first and last name
        names = dg_proj['player'].str.split(expand=True)                  
        names[0] = names[0].str.rstrip(",")
        names[1] = names[1].str.rstrip(",")
        names['player'] = names[1] + " " + names[0]
        # specific player to fix
        names['player'] = np.where(names['player']=='Matt Fitzpatrick', 'Matthew Fitzpatrick', names['player'])
        names['player'] = np.where(names['player']=='Si Kim', 'Si Woo Kim', names['player'])
        names['player'] = np.where(names['player']=='Min Lee', 'Min Woo Lee', names['player'])
        names['player'] = np.where(names['player']=='Byeong An', 'Byeong Hun An', names['player'])
        names['player'] = np.where(names['player']=='Rooyen Van', 'Erik Van Rooyen', names['player'])
        names['player'] = np.where(names['player']=='Vince Whaley', 'Vincent Whaley', names['player'])
        return names.player

# set index to new names
dg_proj.set_index(fix_long_names(dg_proj),inplace=True)

# merge fantrax rosters with datagolf projections
week = pd.merge(teams,dg_proj, left_index=True, right_index=True)#.reset_index()
week[['player','team','active_reserve']] = week[['player','team','active_reserve']].astype('string')
week.sort_values('proj_pts',ascending=False,inplace=True)
week

In [ ]:
sorted(week[week.active_reserve == 'Active'].index)

In [ ]:
####   CURRENT WEEK INPUTS   ####
current_week = 13
tournament = "Valero"
m1 = ['New Team 4','unit_circle']
m2 = ['txmoonshine','Team Gamble']
m3 = ['Putt Pirates','Philly919']
m4 = ['AlphaWired','Sneads Foot']   

In [ ]:
def get_matchup_bar(week,matchup):
        matchup_bar = px.bar(week[(week.active_reserve=='Active') & (week.team.isin(matchup))].drop(columns='player').sort_values(by = 'proj_pts',ascending=False).reset_index(),
                             y = 'proj_pts',
                             color = 'team',
                             color_discrete_map=team_color,
                             labels = {'index':"", 'proj_pts':''},
                             text='player',
                             template = 'plotly_dark',
                             hover_name='proj_pts',
                             height=250,
                             log_y=True
                             ).update_xaxes(showticklabels=False
                             ).update_yaxes(tickvals=[50,60,70,80,90,100], tickfont=dict(color='#5A5856')
                             ).update_yaxes(gridcolor="#B1A999", tickfont=dict(color='#5A5856')
                             ).update_layout(legend=dict(orientation='h',title='',y=1.25,x=.2,font_color='#5A5856'))
        return matchup_bar

In [ ]:
get_matchup_bar(week,m4)

In [ ]:
#  TOP 20 PLAYS
top20 = week.sort_values(by = 'proj_pts',ascending=False)[:20].reset_index(drop=True)
px.bar(top20,
        y = 'proj_pts',
        color = 'team',
        color_discrete_map=team_color,
        labels = {'index':"", 'proj_pts':'Projected Pts'},
        text=week[:20].index,
        #       text_auto = ",.0f",
        template = 'plotly_white',
        height=400,
        log_y=True,
        hover_name='player'
        ).add_hline(y=week.proj_pts.mean(),line_color='darkslategrey'
        ).update_xaxes(showticklabels=False
        ).update_yaxes(showgrid=False)

In [ ]:
# ALL PLAYER PROJECTIONS FOR THE WEEK WITH ACTIVE/INACTIVE CALLOUT

fig = px.bar(week.sort_values(by = 'proj_pts',ascending=False).drop(columns='player').reset_index(),#[:47],
       y = 'proj_pts',
       template='plotly_white',
       title='Arnold Palmer Invitational Players on Rosters',
#        hover_name = 'player',
       color = 'active_reserve',
       color_discrete_map=active_color,height=450,
       labels = {'_index':"", 'proj_pts':'Projected Pts'}).add_hline(y=week.proj_pts.mean()).update_xaxes(showticklabels=False).update_layout(title_x=.5)
fig.show()


In [ ]:
# histogram showing active/inactive

px.histogram(week,
             x='proj_pts',
             opacity=.7,
             nbins=8, 
             color = 'active_reserve',
             color_discrete_map=active_color,
             barmode='group',
             text_auto=True,
             height=400)#.update_xaxes(tickmode='array',tick_vals=["40-49", "50-59", "60-69", "70-79", "80-89"])

In [ ]:
# MAKE DATAFRAME WITH ONLY TOP 6 PROJECTED PLAYERS BY TEAM

top_6_proj = pd.DataFrame()

for team in week.team.unique():
    temp = week[week.team==team][['team','player','proj_pts','active_reserve']].sort_values(by='proj_pts',ascending=False)[:6]
    top_6_proj = pd.concat([top_6_proj,temp])
    top_6_proj = top_6_proj.sort_values('proj_pts',ascending=False).reset_index(drop=True)

print("Of the 48 Top Plays...")
print(top_6_proj.active_reserve.value_counts())
top_6_proj.sort_values('proj_pts',ascending=False).reset_index(drop=True).head()

In [ ]:
teams_dict = {
        '919':'Philly919',
        'u_c':'unit_circle',
        'NT 4':'New Team 4',
        'NT 8':'Sneads Foot',
        'txms':'txmoonshine',
        'MG':'Team Gamble',
        'grrr':'Putt Pirates',
        '[AW]':'AlphaWired'
        }

tab_string = []

for i in range(len(teams_dict.keys())):
    tab_string.append(f"tab{i},")

''.join(tab_string) = st.tabs(list(teams_dict.keys()))

for tab in tab_string:
    for team in list(teams_dict.keys()):
        tab_string[tab].rstrip(",").plotly_chart(get_team_bar(week, team), use_container_width=True, config=config)

In [ ]:
for i in tab_string:
    f'{i.rstrip(",").plotly_chart(get_team_bar(week, team), use_container_width=True, config=config)}'

In [ ]:
# PLAYER OPTIONS FOR EACH TEAM THIS WEEK

for team in week.team.unique():
    fig = px.bar(week[week.team == team].sort_values(by='proj_pts',ascending=False), 
                 x = 'player', 
                 y = 'proj_pts', 
                 height=350,
                 width=600,
                 title = f"Choices for {team}", 
                 color='active_reserve', 
                 text_auto='.2s',
                 template = 'plotly_dark',
                 color_discrete_map=active_color,
                 labels = {'proj_pts':'Projected Points','player':"Eligible Players"})
    fig.show()

In [ ]:
get_matchups(19)